In [4]:
import pandas as pd

# Load stock data and sales data
stock_data = pd.read_csv('stock_data.csv')
sales_data = pd.read_csv('sales_data.csv')

# Summarize total sales per product
sales_summary = sales_data.groupby('ProductID').agg({'UnitsSold': 'sum'}).reset_index()

# Merge stock data with sales summary
inventory_data = pd.merge(stock_data, sales_summary, on='ProductID', how='left')

# Define a simple restocking threshold (e.g., products sold more than 5 units in the past 7 days and stock is low)
def restock_recommendation(row):
    # Assuming a simple rule: if sales in the last period exceed half of the current stock, recommend restock
    sales_velocity = row['UnitsSold']
    stock_level = row['CurrentStock']
    
    # Threshold logic for restocking: if sales velocity is more than half of the current stock, recommend restocking
    if sales_velocity > (stock_level / 2):
        return 'Restock Needed'
    else:
        return 'Stock Sufficient'

# Apply the restock recommendation function
inventory_data['RestockRecommendation'] = inventory_data.apply(restock_recommendation, axis=1)

# Display the results
print("Restock Recommendations:")
print(inventory_data[['ProductID', 'ProductName', 'CurrentStock', 'UnitsSold', 'RestockRecommendation']])

Restock Recommendations:
   ProductID ProductName  CurrentStock  UnitsSold RestockRecommendation
0        101       Apple            50         22      Stock Sufficient
1        102      Banana            30         23        Restock Needed
2        103      Carrot            40         12      Stock Sufficient
3        104      Tomato            30         15      Stock Sufficient
4        105      Potato            25         38        Restock Needed


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox

# Load product data from CSV
def load_data():
    try:
        return pd.read_csv('inventory.csv')
    except FileNotFoundError:
        # Create a new empty dataframe with predefined columns if the CSV doesn't exist
        return pd.DataFrame(columns=["ProductID", "ProductName", "Price", "Stock", "UnitsSold"])

# Save product data to CSV
def save_data(data):
    data.to_csv('inventory.csv', index=False)

# Function to update stock in the system
def update_stock(data, product_id, new_stock):
    data.loc[data['ProductID'] == product_id, 'Stock'] = new_stock
    save_data(data)

# Function to recommend restocking based on sales
def restock_recommendation(data):
    recommendations = []
    for index, row in data.iterrows():
        sales_velocity = row['UnitsSold']
        stock_level = row['Stock']
        
        # If sales exceed half of the stock, recommend restocking
        if sales_velocity > (stock_level / 2):
            recommendations.append(f"Restock {row['ProductName']} (ID: {row['ProductID']})")
    
    return recommendations

# IMS Functions for Add, Update, Delete Products
def add_product(data, product_id, name, price, stock, units_sold):
    # Create a new product row with the correct columns
    new_product = pd.DataFrame({
        "ProductID": [product_id],
        "ProductName": [name],
        "Price": [price],
        "Stock": [stock],
        "UnitsSold": [units_sold]
    })
    
    # Concatenate the new row with the existing data, ensuring the new product is correctly added
    data = pd.concat([data, new_product], ignore_index=True)
    save_data(data)

def delete_product(data, product_id):
    data = data[data['ProductID'] != product_id]
    save_data(data)
    return data

# Function to view all products
def view_products(data):
    return data[['ProductID', 'ProductName', 'Price', 'Stock', 'UnitsSold']]

# GUI for IMS and DSS
class IMS_DSS_App:
    def __init__(self, root):
        self.root = root
        self.root.title("Inventory Management System with DSS")
        
        self.data = load_data()

        self.create_widgets()

    def create_widgets(self):
        # Add Product Section
        self.add_product_label = tk.Label(self.root, text="Add New Product")
        self.add_product_label.grid(row=0, column=0, columnspan=2)

        self.product_id_label = tk.Label(self.root, text="Product ID:")
        self.product_id_label.grid(row=1, column=0, sticky='e')
        self.product_id_entry = tk.Entry(self.root)
        self.product_id_entry.grid(row=1, column=1)

        self.product_name_label = tk.Label(self.root, text="Product Name:")
        self.product_name_label.grid(row=2, column=0, sticky='e')
        self.product_name_entry = tk.Entry(self.root)
        self.product_name_entry.grid(row=2, column=1)

        self.price_label = tk.Label(self.root, text="Price:")
        self.price_label.grid(row=3, column=0, sticky='e')
        self.price_entry = tk.Entry(self.root)
        self.price_entry.grid(row=3, column=1)

        self.stock_label = tk.Label(self.root, text="Stock:")
        self.stock_label.grid(row=4, column=0, sticky='e')
        self.stock_entry = tk.Entry(self.root)
        self.stock_entry.grid(row=4, column=1)

        self.units_sold_label = tk.Label(self.root, text="Units Sold:")
        self.units_sold_label.grid(row=5, column=0, sticky='e')
        self.units_sold_entry = tk.Entry(self.root)
        self.units_sold_entry.grid(row=5, column=1)

        self.add_button = tk.Button(self.root, text="Add Product", command=self.add_product)
        self.add_button.grid(row=6, column=0, columnspan=2)

        # View Products Section
        self.view_button = tk.Button(self.root, text="View Products", command=self.view_products)
        self.view_button.grid(row=7, column=0, columnspan=2)

        # Restock Recommendation Section
        self.restock_button = tk.Button(self.root, text="Get Restock Recommendations", command=self.get_restock_recommendations)
        self.restock_button.grid(row=8, column=0, columnspan=2)

        # Display Area
        self.display_area = tk.Label(self.root, text="", anchor='w', justify='left')
        self.display_area.grid(row=9, column=0, columnspan=2)

    def add_product(self):
        product_id = self.product_id_entry.get()
        product_name = self.product_name_entry.get()
        price = float(self.price_entry.get())
        stock = int(self.stock_entry.get())
        units_sold = int(self.units_sold_entry.get())

        add_product(self.data, product_id, product_name, price, stock, units_sold)
        messagebox.showinfo("Success", "Product added successfully!")

    def view_products(self):
        products = view_products(self.data)
        display_text = products.to_string(index=False)
        self.display_area.config(text=display_text)

    def get_restock_recommendations(self):
        recommendations = restock_recommendation(self.data)
        if recommendations:
            recommendations_text = "\n".join(recommendations)
            self.display_area.config(text=recommendations_text)
        else:
            self.display_area.config(text="No restock needed at the moment.")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = IMS_DSS_App(root)
    root.mainloop()
